In [1]:
pip install pandas numpy scikit-learn Pillow Keras_tuner

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, LSTM, Dense, Dropout, BatchNormalization, Reshape
from keras.applications import ResNet152
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
import keras_tuner as kt
from keras_tuner import BayesianOptimization, HyperParameters
from keras.callbacks import EarlyStopping
from keras_tuner.engine.hypermodel import HyperModel

In [7]:

# Define the root directory where your image folders are located
root_directory = "PartB_DFU_dataset - Copy"

# Initialize lists to store image paths and corresponding class labels for both datasets
image_paths_ischaemia = []
categories_ischaemia = []
image_paths_infection = []
categories_infection = []

# Iterate over each class and its subdirectories
for class_name in ["Infection", "Ischaemia"]:
    for augmentation_type in ["Aug-Negative", "Aug-Positive"]:
        folder_path = os.path.join(root_directory, class_name, augmentation_type)
        category = f"{class_name.lower()}{'pov' if 'Positive' in augmentation_type else 'neg'}"
        
        # Iterate over image files in the current directory
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".jpg"):  # Assuming images are jpg format
                image_path = os.path.join(folder_path, file_name)
                if class_name == "Ischaemia":
                    image_paths_ischaemia.append(image_path)
                    categories_ischaemia.append("ischemia" if "Positive" in augmentation_type else "non-ischemia")
                elif class_name == "Infection":
                    image_paths_infection.append(image_path)
                    categories_infection.append("infection" if "Positive" in augmentation_type else "non-infection")

# Create DataFrames for each dataset
df_ischaemia = pd.DataFrame({"category": categories_ischaemia, "image_path": image_paths_ischaemia})
df_infection = pd.DataFrame({"category": categories_infection, "image_path": image_paths_infection})

# Label encoding for Ischaemia dataset
label_encoder_ischaemia = LabelEncoder()
df_ischaemia['Class_Label'] = label_encoder_ischaemia.fit_transform(df_ischaemia['category'])
print("Ischaemia Class Mapping:")
for class_label, numerical_label in zip(df_ischaemia['category'].unique(), df_ischaemia['Class_Label'].unique()):
    print(f"{class_label}: {numerical_label}")

# Label encoding for Infection dataset
label_encoder_infection = LabelEncoder()
df_infection['Class_Label'] = label_encoder_infection.fit_transform(df_infection['category'])
print("Infection Class Mapping:")
for class_label, numerical_label in zip(df_infection['category'].unique(), df_infection['Class_Label'].unique()):
    print(f"{class_label}: {numerical_label}")

# Shuffle both DataFrames
df_ischaemia = df_ischaemia.sample(frac=1).reset_index(drop=True)
df_infection = df_infection.sample(frac=1).reset_index(drop=True)

# Helper function to load and process images
def load_images(df):
    images = []
    target_labels = []   
    for index, row in df.iterrows():
        image = Image.open(row['image_path']).convert('RGB')
        image_array = np.array(image.resize((256, 256)))  # Resize image to fit DenseNet input size
        images.append(image_array)
        target_labels.append(row['Class_Label'])
    return np.array(images), np.array(target_labels)

# Load images for both datasets
images_ischaemia, target_labels_ischaemia = load_images(df_ischaemia)
images_infection, target_labels_infection = load_images(df_infection)

print("Shape of Ischaemia images array:", images_ischaemia.shape)
print("Shape of Ischaemia target labels array:", target_labels_ischaemia.shape)
print("Shape of Infection images array:", images_infection.shape)
print("Shape of Infection target labels array:", target_labels_infection.shape)

# Split the Ischaemia dataset
X_train_ischaemia, X_test_ischaemia, y_train_ischaemia, y_test_ischaemia = train_test_split(
    images_ischaemia, target_labels_ischaemia, test_size=0.3, random_state=42)
X_val_ischaemia, X_test_ischaemia, y_val_ischaemia, y_test_ischaemia = train_test_split(
    X_test_ischaemia, y_test_ischaemia, test_size=0.25, random_state=42)  # 0.25 * 0.3 = 0.075

# Split the Infection dataset
X_train_infection, X_test_infection, y_train_infection, y_test_infection = train_test_split(
    images_infection, target_labels_infection, test_size=0.3, random_state=42)
X_val_infection, X_test_infection, y_val_infection, y_test_infection = train_test_split(
    X_test_infection, y_test_infection, test_size=0.25, random_state=42)  # 0.25 * 0.3 = 0.075

print("Ischaemia Training set shape:", X_train_ischaemia.shape, y_train_ischaemia.shape)
print("Ischaemia Validation set shape:", X_val_ischaemia.shape, y_val_ischaemia.shape)
print("Ischaemia Test set shape:", X_test_ischaemia.shape, y_test_ischaemia.shape)
print("Infection Training set shape:", X_train_infection.shape, y_train_infection.shape)
print("Infection Validation set shape:", X_val_infection.shape, y_val_infection.shape)
print("Infection Test set shape:", X_test_infection.shape, y_test_infection.shape)


Ischaemia Class Mapping:
non-ischemia: 1
ischemia: 0
Infection Class Mapping:
non-infection: 1
infection: 0
Shape of Ischaemia images array: (9870, 256, 256, 3)
Shape of Ischaemia target labels array: (9870,)
Shape of Infection images array: (5890, 256, 256, 3)
Shape of Infection target labels array: (5890,)
Ischaemia Training set shape: (6909, 256, 256, 3) (6909,)
Ischaemia Validation set shape: (2220, 256, 256, 3) (2220,)
Ischaemia Test set shape: (741, 256, 256, 3) (741,)
Infection Training set shape: (4123, 256, 256, 3) (4123,)
Infection Validation set shape: (1325, 256, 256, 3) (1325,)
Infection Test set shape: (442, 256, 256, 3) (442,)


In [8]:

# Define the HyperModel class for Bayesian optimization using ResNet152
class MyHyperModel(HyperModel):

    def __init__(self):
        self.base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
        for layer in self.base_model.layers:
            layer.trainable = False

    def build(self, hp):
        model = Sequential([
            self.base_model,
            GlobalAveragePooling2D(),
            Reshape((8, 256)),  # Adjust the reshape dimensions based on the output size of ResNet152
            LSTM(
                units=hp.Int('units_1', min_value=8, max_value=64, step=8),
                return_sequences=True,
                kernel_regularizer=l2(hp.Float('l2_1', min_value=0.001, max_value=0.01, step=0.001))
            ),
            Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)),
            LSTM(
                units=hp.Int('units_2', min_value=8, max_value=64, step=8),
                return_sequences=True,
                kernel_regularizer=l2(hp.Float('l2_2', min_value=0.001, max_value=0.01, step=0.001))
            ),
            Dropout(hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)),
            LSTM(
                units=hp.Int('units_3', min_value=8, max_value=64, step=8),
                kernel_regularizer=l2(hp.Float('l2_3', min_value=0.001, max_value=0.01, step=0.001))
            ),
            Dropout(hp.Float('dropout_3', min_value=0.1, max_value=0.5, step=0.1)),
            BatchNormalization(),
            Dense(
                units=hp.Int('dense_units', min_value=8, max_value=64, step=8),
                activation='relu',
                kernel_regularizer=l2(hp.Float('l2_dense', min_value=0.001, max_value=0.01, step=0.001))
            ),
            Dropout(hp.Float('dropout_dense', min_value=0.1, max_value=0.5, step=0.1)),
            BatchNormalization(),
            Dense(3, activation='softmax', kernel_regularizer=l2(hp.Float('l2_final', min_value=0.001, max_value=0.01, step=0.001)))
        ])

        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        
        return model

In [9]:
tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_accuracy',
    max_trials=5,
    directory='my_dir',
    project_name='resnet152_optimization'
)

2024-07-30 07:32:04.861440: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22066 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9
2024-07-30 07:32:04.863635: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22066 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:21:00.0, compute capability: 8.9


In [10]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
tuner.search(X_train_ischaemia, y_train_ischaemia, epochs=50, validation_data=(X_val_ischaemia, y_val_ischaemia),callbacks=[reduce_lr, early_stopping])

Trial 5 Complete [00h 07m 17s]
val_accuracy: 0.9594594836235046

Best val_accuracy So Far: 0.9720720648765564
Total elapsed time: 00h 49m 00s


In [12]:
best_model = tuner.get_best_models(num_models=1)[0]
test_loss, test_accuracy = best_model.evaluate(X_test_ischaemia, y_test_ischaemia)
print("Test accuracy:", test_accuracy)

24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step - accuracy: 0.9756 - loss: 0.1720
Test accuracy: 0.9689608812332153
